In [5]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# Data Munging

## Relational Data

The simplest type of data we have see might consist a single table with a some columns and some rows. This sort of data is easy to analyze and compute and we generally want to reduce our data to a single table before we start running machine learning algorithms. Yet, real world data doesn't necessarily fit into this paradigm.  Most real world data is messy and complicated which doesn't fit well into a tabular format and we will have to do some work to reduce this complexity.  Additionally, in many case we can reduce our memory cost by not keeping data in a single table, but instead in a set of data structures with defined relations between them. 

Here we will explore a bit of data and see how combining different sets of data can help us generate useful features.

First we need some data.  We will make use of some data from Wikipedia and we will use the pandas `read_html` function to scrape the data from a particular webpage.  We will study the top 10 companies in the Fortune Global 500 which conveniently have [their own Wikipedia page](https://en.wikipedia.org/w/index.php?title=Fortune_Global_500&oldid=855890446).

We will download the data in tabular form, but work with it as a list of dictionaries, this will allow us to get used to working with unstructured data.

In [6]:
import requests

In [7]:
response = requests.get('https://en.wikipedia.org/w/index.php?title=Fortune_Global_500&oldid=855890446')

# dir(response)

In [8]:
# dir(response.text)

In [9]:
from bs4 import BeautifulSoup

In [10]:
soup = BeautifulSoup(response.text, 'lxml')

In [11]:
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Fortune Global 500 - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XeJJYApAIC8AAFArHQoAAACN","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Fortune_Global_500","wgTitle":"Fortune Global 500","wgCurRevisionId":927551890,"wgRevisionId":855890446,"wgArticleId":17581425,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["All articles with unsourced statements","Articles with unsourced sta

In [12]:
soup.find_all('table')

[<table border="1" class="wikitable sortable">
 <tbody><tr>
 <th>Rank
 </th>
 <th>Company
 </th>
 <th>Country
 </th>
 <th>Industry
 </th>
 <th>Revenue in USD
 </th></tr>
 <tr>
 <td>1
 </td>
 <td><a href="/wiki/Walmart" title="Walmart">Walmart</a>
 </td>
 <td><span class="flagicon"><img alt="" class="thumbborder" data-file-height="650" data-file-width="1235" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/23px-Flag_of_the_United_States.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/35px-Flag_of_the_United_States.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/46px-Flag_of_the_United_States.svg.png 2x" width="23"/> </span><a href="/wiki/United_States" title="United States">United States</a>
 </td>
 <td>Retail
 </td>
 <td>$500 billion
 </td></tr>
 <tr>
 <td>2
 </td>
 <td><a class="mw-redirect" href="/wiki/State_Grid" title="State Grid">St

In [13]:
soup.find_all('table')[0].text

'\n\nRank\n\nCompany\n\nCountry\n\nIndustry\n\nRevenue in USD\n\n\n1\n\nWalmart\n\n\xa0United States\n\nRetail\n\n$500 billion\n\n\n2\n\nState Grid\n\n\xa0China\n\nPower\n\n$349 billion\n\n\n3\n\nSinopec Group\n\n\xa0China\n\nPetroleum\n\n$327 billion\n\n\n4\n\nChina National Petroleum\n\n\xa0China\n\nPetroleum\n\n$326 billion\n\n\n5\n\nRoyal Dutch Shell\n\n\xa0Netherlands\n\nPetroleum\n\n$312 billion\n\n\n6\n\nToyota Motor\n\n\xa0Japan\n\nAutomobiles\n\n$265 billion\n\n\n7\n\nVolkswagen\n\n\xa0Germany\n\nAutomobiles\n\n$260 billion\n\n\n8\n\nBP\n\n\xa0United Kingdom\n\nPetroleum\n\n$245 billion\n\n\n9\n\nExxon Mobil\n\n\xa0United States\n\nPetroleum\n\n$244 billion\n\n\n10\n\nBerkshire Hathaway\n\n\xa0United States\n\nProducts\n\n$242 billion\n'

In [14]:
import pandas as pd
import json
df = pd.read_html('https://en.wikipedia.org/w/index.php?title=Fortune_Global_500&oldid=855890446', header=0)[0]
fortune_500 = json.loads(df.to_json(orient="records"))
df

,Rank,Company,Country,Industry,Revenue in USD
0,1,Walmart,United States,Retail,$500 billion
1,2,State Grid,China,Power,$349 billion
2,3,Sinopec Group,China,Petroleum,$327 billion
3,4,China National Petroleum,China,Petroleum,$326 billion
4,5,Royal Dutch Shell,Netherlands,Petroleum,$312 billion
5,6,Toyota Motor,Japan,Automobiles,$265 billion
6,7,Volkswagen,Germany,Automobiles,$260 billion
7,8,BP,United Kingdom,Petroleum,$245 billion
8,9,Exxon Mobil,United States,Petroleum,$244 billion
9,10,Berkshire Hathaway,United States,Products,$242 billion


Lets look at the data.

In [15]:
fortune_500

[{'Rank': 1,
  'Company': 'Walmart',
  'Country': 'United States',
  'Industry': 'Retail',
  'Revenue in USD': '$500 billion'},
 {'Rank': 2,
  'Company': 'State Grid',
  'Country': 'China',
  'Industry': 'Power',
  'Revenue in USD': '$349 billion'},
 {'Rank': 3,
  'Company': 'Sinopec Group',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Revenue in USD': '$327 billion'},
 {'Rank': 4,
  'Company': 'China National Petroleum',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Revenue in USD': '$326 billion'},
 {'Rank': 5,
  'Company': 'Royal Dutch Shell',
  'Country': 'Netherlands',
  'Industry': 'Petroleum',
  'Revenue in USD': '$312 billion'},
 {'Rank': 6,
  'Company': 'Toyota Motor',
  'Country': 'Japan',
  'Industry': 'Automobiles',
  'Revenue in USD': '$265 billion'},
 {'Rank': 7,
  'Company': 'Volkswagen',
  'Country': 'Germany',
  'Industry': 'Automobiles',
  'Revenue in USD': '$260 billion'},
 {'Rank': 8,
  'Company': 'BP',
  'Country': 'United Kingdom',
  'Industry': 'Petr

This is a great start to our analysis, however, there really isn't that much information here, we will need to bring in additional data sources to get any further understanding of these companies.

The first question we might want to ask is how many employees does it take to get that revenue, in other words, what is the revenue per employee?  Luckily, we can use Wikipedia to get that data as well, we have scraped this data manually (all from Wikipedia) and created the following dictionary.

In [16]:
other_data = [
    {"name": "Walmart",
     "employees": 2300000,
     "year founded": 1962
    },
    {"name": "State Grid Corporation of China",
     "employees": 927839,
     "year founded": 2002},
    {"name": "China Petrochemical Corporation",
     "employees":358571,
     "year founded": 1998
     },
    {"name": "China National Petroleum Corporation",
     "employees": 1636532,
     "year founded": 1988},
    {"name": "Toyota Motor Corporation",
     "employees": 364445,
     "year founded": 1937},
    {"name": "Volkswagen AG",
     "employees": 642292,
     "year founded": 1937},
    {"name": "Royal Dutch Shell",
     "employees": 92000,
     "year founded": 1907},
    {"name": "Berkshire Hathaway Inc.",
     "employees":377000,
     "year founded": 1839},
    {"name": "Apple Inc.",
     "employees": 123000,
     "year founded": 1976},
    {"name": "Exxon Mobile Corporation",
     "employees": 69600,
     "year founded": 1999},
    {"name": "BP plc",
     "employees": 74000,
     "year founded": 1908}
]

Some data have a slightly different name than in our original set, so we will keep a dictionary of mappings between the two.  Notice, we only include the mapping in the dictionary if there is a difference.

In [17]:
mapping = {
    'Apple': 'Apple Inc.',
    'BP': 'BP plc',
    'Berkshire Hathaway': 'Berkshire Hathaway Inc.',
    'China National Petroleum': 'China National Petroleum Corporation',
    'Exxon Mobil': 'Exxon Mobile Corporation',
    'Sinopec Group': 'China Petrochemical Corporation',
    'State Grid': 'State Grid Corporation of China',
    'Toyota Motor': 'Toyota Motor Corporation',
    'Volkswagen': 'Volkswagen AG'
}

This data is one to one, meaning the data contained in one source only aligns with a single element in the other source, thus we should be able to put these together.  However, we know that the data isn't in a great form to be joined at the moment.  This is for two reasons

1. All the names will not align (we need to use our mapping)
2. The `list` structure is not optimized for looking through elements. 

While for 10 elements the second reason won't really matter, for larger data sets such performance considerations are extremely important.  We can turn this list of dictionaries into a dictionary of dictionaries, so we can quickly access each element of the data.

In [18]:
dict_data = {k["name"] : k for k in other_data}
dict_data

{'Walmart': {'name': 'Walmart', 'employees': 2300000, 'year founded': 1962},
 'State Grid Corporation of China': {'name': 'State Grid Corporation of China',
  'employees': 927839,
  'year founded': 2002},
 'China Petrochemical Corporation': {'name': 'China Petrochemical Corporation',
  'employees': 358571,
  'year founded': 1998},
 'China National Petroleum Corporation': {'name': 'China National Petroleum Corporation',
  'employees': 1636532,
  'year founded': 1988},
 'Toyota Motor Corporation': {'name': 'Toyota Motor Corporation',
  'employees': 364445,
  'year founded': 1937},
 'Volkswagen AG': {'name': 'Volkswagen AG',
  'employees': 642292,
  'year founded': 1937},
 'Royal Dutch Shell': {'name': 'Royal Dutch Shell',
  'employees': 92000,
  'year founded': 1907},
 'Berkshire Hathaway Inc.': {'name': 'Berkshire Hathaway Inc.',
  'employees': 377000,
  'year founded': 1839},
 'Apple Inc.': {'name': 'Apple Inc.',
  'employees': 123000,
  'year founded': 1976},
 'Exxon Mobile Corporatio

**Question:** If we had many entries in `other_data`, we could display a small piece by printing `other_data[:5]`. With dataframes we might use `df.head()`. Can you think of a way to print out a small piece of a dictionary?

In [19]:
# d represents the dictionary whose head is to be printed
# n represents the the number of items to be printed starting from the first item, like head(5)
def dict_head(d, n):
    return dict(list(d.items())[:n])

In [20]:
dict_head(dict_data, 1)

{'Walmart': {'name': 'Walmart', 'employees': 2300000, 'year founded': 1962}}

In [21]:
dict_head(dict_data, 3)

{'Walmart': {'name': 'Walmart', 'employees': 2300000, 'year founded': 1962},
 'State Grid Corporation of China': {'name': 'State Grid Corporation of China',
  'employees': 927839,
  'year founded': 2002},
 'China Petrochemical Corporation': {'name': 'China Petrochemical Corporation',
  'employees': 358571,
  'year founded': 1998}}

Now we can easily compute the revenue per employee, we need to map the "Company" value in our original data with the "name" column of this other data, but we also need to use the mapping to ensure the columns will line up.  We in general don't want to mutate our original data, so lets make a new list of dictionaries with this new feature (revenue per employee).  On the course of doing this, we will need to handle converting some numbers like `$500 Billion` to a numeric value.  Lets create a function to do this.

In [22]:
def convert_revenue(x):
    return float(x.lstrip('$').rstrip('billion')) * 1e9

assert convert_revenue('$500 billion') == 500e9

In [23]:
foo = '$500b billion'
foo.lstrip('$').rstrip('b billion')

'500'

Now we should be able to create a few functions to compute this revenue per employee and create a data list.

In [24]:
fortune_500

[{'Rank': 1,
  'Company': 'Walmart',
  'Country': 'United States',
  'Industry': 'Retail',
  'Revenue in USD': '$500 billion'},
 {'Rank': 2,
  'Company': 'State Grid',
  'Country': 'China',
  'Industry': 'Power',
  'Revenue in USD': '$349 billion'},
 {'Rank': 3,
  'Company': 'Sinopec Group',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Revenue in USD': '$327 billion'},
 {'Rank': 4,
  'Company': 'China National Petroleum',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Revenue in USD': '$326 billion'},
 {'Rank': 5,
  'Company': 'Royal Dutch Shell',
  'Country': 'Netherlands',
  'Industry': 'Petroleum',
  'Revenue in USD': '$312 billion'},
 {'Rank': 6,
  'Company': 'Toyota Motor',
  'Country': 'Japan',
  'Industry': 'Automobiles',
  'Revenue in USD': '$265 billion'},
 {'Rank': 7,
  'Company': 'Volkswagen',
  'Country': 'Germany',
  'Industry': 'Automobiles',
  'Revenue in USD': '$260 billion'},
 {'Rank': 8,
  'Company': 'BP',
  'Country': 'United Kingdom',
  'Industry': 'Petr

In [25]:
def rev_per_emp(company):
    name = company[u'Company']
    n_employees = dict_data[mapping.get(name, name)].get('employees')
    company['rev per emp'] = convert_revenue(company[u'Revenue in USD'])/n_employees
    return company

def make_copy(d):
    return {k:v for k,v in d.items()}

data = list(map(lambda company : rev_per_emp(make_copy(company)), fortune_500))

# my_data = list(map(lambda company: rev_per_emp(company), fortune_500)) # this will mutate the fortune_500 data

Lets take a look at our new data and also the old data to ensure we didn't mutate anything.

In [26]:
data[:2]

[{'Rank': 1,
  'Company': 'Walmart',
  'Country': 'United States',
  'Industry': 'Retail',
  'Revenue in USD': '$500 billion',
  'rev per emp': 217391.30434782608},
 {'Rank': 2,
  'Company': 'State Grid',
  'Country': 'China',
  'Industry': 'Power',
  'Revenue in USD': '$349 billion',
  'rev per emp': 376142.84374767606}]

In [27]:
fortune_500[:2]

[{'Rank': 1,
  'Company': 'Walmart',
  'Country': 'United States',
  'Industry': 'Retail',
  'Revenue in USD': '$500 billion'},
 {'Rank': 2,
  'Company': 'State Grid',
  'Country': 'China',
  'Industry': 'Power',
  'Revenue in USD': '$349 billion'}]

Now we can sort these values.  We first can select out on the elements we care about and then sort that list.

In [28]:
rev_per_emp = sorted([(i[u'Company'], i['rev per emp']) for i in data], 
                   key=lambda x : x[1],
                   reverse=True)
rev_per_emp

[('Exxon Mobil', 3505747.1264367816),
 ('Royal Dutch Shell', 3391304.347826087),
 ('BP', 3310810.810810811),
 ('Sinopec Group', 911953.2812190612),
 ('Toyota Motor', 727133.0379069544),
 ('Berkshire Hathaway', 641909.8143236075),
 ('Volkswagen', 404800.30889377417),
 ('State Grid', 376142.84374767606),
 ('Walmart', 217391.30434782608),
 ('China National Petroleum', 199201.72657790987)]

This results in a much different order.  What does this tell us about the companies?

Now lets pull in some other data (this is data science, more data is always better!).  We can see that these companies are in a few different industries, let find out which ones.

In [29]:
data

[{'Rank': 1,
  'Company': 'Walmart',
  'Country': 'United States',
  'Industry': 'Retail',
  'Revenue in USD': '$500 billion',
  'rev per emp': 217391.30434782608},
 {'Rank': 2,
  'Company': 'State Grid',
  'Country': 'China',
  'Industry': 'Power',
  'Revenue in USD': '$349 billion',
  'rev per emp': 376142.84374767606},
 {'Rank': 3,
  'Company': 'Sinopec Group',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Revenue in USD': '$327 billion',
  'rev per emp': 911953.2812190612},
 {'Rank': 4,
  'Company': 'China National Petroleum',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Revenue in USD': '$326 billion',
  'rev per emp': 199201.72657790987},
 {'Rank': 5,
  'Company': 'Royal Dutch Shell',
  'Country': 'Netherlands',
  'Industry': 'Petroleum',
  'Revenue in USD': '$312 billion',
  'rev per emp': 3391304.347826087},
 {'Rank': 6,
  'Company': 'Toyota Motor',
  'Country': 'Japan',
  'Industry': 'Automobiles',
  'Revenue in USD': '$265 billion',
  'rev per emp': 727133.037906

In [30]:
from collections import Counter
Counter(i[u'Industry'] for i in data)

Counter({'Retail': 1,
         'Power': 1,
         'Petroleum': 5,
         'Automobiles': 2,
         'Products': 1})

One thing we might want to know is what sort of market share they have of the specific industry to which they belong.  Let's look at the two industries that categorize the 6 of the top 10, `Automobiles` and `Petroleum`.  We can select only those elements of our data to work with.

In [31]:
sub_data = [i for i in data if i[u'Industry'] in [u'Automobiles', u'Petroleum']]
sub_data

[{'Rank': 3,
  'Company': 'Sinopec Group',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Revenue in USD': '$327 billion',
  'rev per emp': 911953.2812190612},
 {'Rank': 4,
  'Company': 'China National Petroleum',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Revenue in USD': '$326 billion',
  'rev per emp': 199201.72657790987},
 {'Rank': 5,
  'Company': 'Royal Dutch Shell',
  'Country': 'Netherlands',
  'Industry': 'Petroleum',
  'Revenue in USD': '$312 billion',
  'rev per emp': 3391304.347826087},
 {'Rank': 6,
  'Company': 'Toyota Motor',
  'Country': 'Japan',
  'Industry': 'Automobiles',
  'Revenue in USD': '$265 billion',
  'rev per emp': 727133.0379069544},
 {'Rank': 7,
  'Company': 'Volkswagen',
  'Country': 'Germany',
  'Industry': 'Automobiles',
  'Revenue in USD': '$260 billion',
  'rev per emp': 404800.30889377417},
 {'Rank': 8,
  'Company': 'BP',
  'Country': 'United Kingdom',
  'Industry': 'Petroleum',
  'Revenue in USD': '$245 billion',
  'rev per emp': 3310810

It might be the case that the each particular category has a different relevant metric for market share.  For example, we could look at total revenue for a car company or we could look at cars produced.  

So for the automobile industry we will look at the percent total of cars produced.  We can get this data again from Wikipedia.

In [32]:
df_list = pd.read_html("https://en.wikipedia.org/w/index.php?title=Automotive_industry&oldid=875776152", header=0)
car_totals = json.loads(df_list[0].to_json(orient="records"))
car_by_man = json.loads(df_list[2].to_json(orient='records'))

In [33]:
car_totals[:2]

[{'Year': 1997, 'Production': 54434000, 'Change': '—', 'Source': '[17]'},
 {'Year': 1998, 'Production': 52987000, 'Change': '2.7%', 'Source': '[17]'}]

In [34]:
car_by_man[:2]

[{'Rank': 1, 'Group': 'Toyota', 'Country': 'Japan', 'Vehicles': 10213486},
 {'Rank': 2,
  'Group': 'Volkswagen Group',
  'Country': 'Germany',
  'Vehicles': 10126281}]

Now lets get only the groups we care about and divide by the total production which we will take as the latest year.

In [35]:
total_prod = sorted((i[u"Year"], i[u'Production']) for i in car_totals)[-1][1]
total_prod

97302534

Now we can find the market share for each of the car companies. We will keep track of a market share dictionary.  We will again need to keep track of some slight name differences.

In [36]:
def convert_prod(s):
    return s.replace(',', '').split(' ')[0]

convert_prod('54,434,000 [50]')

'54434000'

In [37]:
list(zip(['Toyota', 'Volkswagen Group'], ['Toyota', 'Volkswagen']))

[('Toyota', 'Toyota'), ('Volkswagen Group', 'Volkswagen')]

In [38]:
car_by_man_dict = {i[u'Group']:i[u'Vehicles'] for i in car_by_man}
market_share = {}
for name, orig_name in zip(['Toyota', 'Volkswagen Group'], ['Toyota', 'Volkswagen']):
    market_share[orig_name] = car_by_man_dict[name]/ float(total_prod)
    
market_share

{'Toyota': 0.1049662899837737, 'Volkswagen': 0.10407006460900597}

Now we can do the same for the Petroleum industry, but in this case, lets compute the market share by revenue.  On Wikipedia, we can find a list of oil companies by revenue.  Although its not a complete list, it has enough companies that we don't expect the companies left off the list to contribute greatly to our analysis.

In [39]:
rev_raw = pd.read_html("https://en.wikipedia.org/w/index.php?title=List_of_largest_oil_and_gas_companies_by_revenue&oldid=871711850", header=1)[0]
rev_raw

,Country,Company name,Revenue 2017(US$ billion)
0,NaN,Saudi Aramco,465.49
1,NaN,Sinopec Group,448.00
2,NaN,China National Petroleum Corporation,428.62
3,NaN,Exxon Mobil,268.9
4,NaN,Royal Dutch Shell,265
5,NaN,Kuwait Petroleum Corporation,251.94
6,NaN,BP,222.8
7,NaN,Total SA,212
8,NaN,Lukoil,144.17
9,NaN,Eni,131.82


In [40]:
rev = rev_raw.iloc[:, 1:3]
rev

,Company name,Revenue 2017(US$ billion)
0,Saudi Aramco,465.49
1,Sinopec Group,448.00
2,China National Petroleum Corporation,428.62
3,Exxon Mobil,268.9
4,Royal Dutch Shell,265
5,Kuwait Petroleum Corporation,251.94
6,BP,222.8
7,Total SA,212
8,Lukoil,144.17
9,Eni,131.82


In [41]:
rev.columns = ['Company', 'Revenue']

rev

,Company,Revenue
0,Saudi Aramco,465.49
1,Sinopec Group,448.00
2,China National Petroleum Corporation,428.62
3,Exxon Mobil,268.9
4,Royal Dutch Shell,265
5,Kuwait Petroleum Corporation,251.94
6,BP,222.8
7,Total SA,212
8,Lukoil,144.17
9,Eni,131.82


In [42]:
rev_cleaned_up = rev[~(rev['Company'] == 'Company name')]

rev_cleaned_up

,Company,Revenue
0,Saudi Aramco,465.49
1,Sinopec Group,448.00
2,China National Petroleum Corporation,428.62
3,Exxon Mobil,268.9
4,Royal Dutch Shell,265
5,Kuwait Petroleum Corporation,251.94
6,BP,222.8
7,Total SA,212
8,Lukoil,144.17
9,Eni,131.82


In [43]:
oil_data = json.loads(rev_cleaned_up.to_json(orient="records"))
oil_data[:2]

[{'Company': 'Saudi Aramco', 'Revenue': '465.49'},
 {'Company': 'Sinopec Group', 'Revenue': '448.00'}]

Now we can compute the totals and market share.  Since the data here might be slightly different (perhaps older) than our original data, we will compute the market share of each company within this data set, then pull out the numbers we care about.

In [44]:
total = sum([float(i[u'Revenue'].rstrip('*')) for i in oil_data])
total

5482.090000000001

In [45]:
shares = {i[u'Company']:float(i[u'Revenue'].rstrip('*'))/total for i in oil_data}

dict_head(shares, 5)   # Recall the dict_head() function we created earlier for visualizing the first n items of a dict

{'Saudi Aramco': 0.08491104669934275,
 'Sinopec Group': 0.0817206576323993,
 'China National Petroleum Corporation': 0.07818550954106918,
 'Exxon Mobil': 0.0490506357976611,
 'Royal Dutch Shell': 0.04833922828702191}

Now we can pull out the companies we care about in the petroleum industry.

In [46]:
petro_companies = [i[u'Company'] for i in data if i['Industry'] == u'Petroleum']
petro_companies

['Sinopec Group',
 'China National Petroleum',
 'Royal Dutch Shell',
 'BP',
 'Exxon Mobil']

Lets check if these are all in the our shares dictionary.

In [47]:
[(i, i in shares) for i in petro_companies]

[('Sinopec Group', True),
 ('China National Petroleum', False),
 ('Royal Dutch Shell', True),
 ('BP', True),
 ('Exxon Mobil', True)]

Some of these companies are directly there, and looking through our dictionary, we can see the others are there without exact names.

In [48]:
dict_head(shares, 5)

{'Saudi Aramco': 0.08491104669934275,
 'Sinopec Group': 0.0817206576323993,
 'China National Petroleum Corporation': 0.07818550954106918,
 'Exxon Mobil': 0.0490506357976611,
 'Royal Dutch Shell': 0.04833922828702191}

In [49]:
shares.keys()

dict_keys(['Saudi Aramco', 'Sinopec Group', 'China National Petroleum Corporation', 'Exxon Mobil', 'Royal Dutch Shell', 'Kuwait Petroleum Corporation', 'BP', 'Total SA', 'Lukoil', 'Eni', 'Valero Energy', 'Chevron Corporation', 'PDVSA', 'Pemex', 'National Iranian Oil Company', 'Gazprom', 'Phillips 66', 'Petronas', 'China National Offshore Oil', 'Marathon Petroleum', 'PTT', 'Rosneft', 'JX Holdings', 'Engie', 'Petrobras', 'Equinor', 'Indian Oil Corporation', 'Sonatrach', 'Reliance Industries', 'Pertamina', 'SOCAR', 'GS Caltex', 'Enterprise Products', 'Repsol', 'Centrica', 'Bharat Petroleum', 'OMV Group', 'Idemitsu Kosan', 'Hindustan Petroleum', 'PKN Orlen', 'Schlumberger', 'Suncor Energy', 'Hellenic Petroleum', 'ConocoPhillips', 'Motor Oil Hellas', 'Ecopetrol', 'CEPSA', 'MOL'])

So lets make a fuzzy match, this will be a pretty simple one where it will try to match words in a name and take the maximum number of matches.

In [50]:
petro_companies

['Sinopec Group',
 'China National Petroleum',
 'Royal Dutch Shell',
 'BP',
 'Exxon Mobil']

In [51]:
words = set(petro_companies[1].split(' '))
words

{'China', 'National', 'Petroleum'}

In [52]:
split_names = {i: set(i.split(' ')) for i in shares.keys()}
dict_head(split_names, 5)

{'Saudi Aramco': {'Aramco', 'Saudi'},
 'Sinopec Group': {'Group', 'Sinopec'},
 'China National Petroleum Corporation': {'China',
  'Corporation',
  'National',
  'Petroleum'},
 'Exxon Mobil': {'Exxon', 'Mobil'},
 'Royal Dutch Shell': {'Dutch', 'Royal', 'Shell'}}

In [53]:
s1 = {'a', 'b', 'c'}
s2 = {'d', 'e', 'c'}
s2.intersection(s1)

{'c'}

In [54]:
len(s2.intersection(s1))

1

In [55]:
res = {k: len(words.intersection(v)) for k, v in split_names.items()}
res2 = list(res.items())
max(res2, key=lambda x: x[1])[0]

'China National Petroleum Corporation'

In [56]:
def fuzzy_match(word, s):
    words = set(word.split(' '))
    overlaps = [(k, len(v.intersection(words))) for k, v in s.items()]
    return max(overlaps, key=lambda x : x[1])[0]

In [57]:
split_names = {i: set(i.split(' ')) for i in shares.keys()}
for i in petro_companies:
    match = fuzzy_match(i, split_names)
    print(f"matched {i} to {match}")
    market_share[i] = shares[match]

matched Sinopec Group to Sinopec Group
matched China National Petroleum to China National Petroleum Corporation
matched Royal Dutch Shell to Royal Dutch Shell
matched BP to BP
matched Exxon Mobil to Exxon Mobil


In [58]:
market_share

{'Toyota': 0.1049662899837737,
 'Volkswagen': 0.10407006460900597,
 'Sinopec Group': 0.0817206576323993,
 'China National Petroleum': 0.07818550954106918,
 'Royal Dutch Shell': 0.04833922828702191,
 'BP': 0.040641434197541446,
 'Exxon Mobil': 0.0490506357976611}

## By industry
We have some nice examples of data munging, now lets see an example of keeping data in a relational fashion.  Lets say we want to add another feature which is the growth of each industry.  If we were to store this data as a single quantity, we would be saving a bunch of extra information, we would be much better off extracting this information and keeping it in a single table so we are not replicating by industry.

## With Pandas

Now we can also perform these same computations with Pandas, lets see how this compares.

In [64]:
df = pd.read_html('https://en.wikipedia.org/w/index.php?title=Fortune_Global_500&oldid=855890446', header=0)[0]
df

,Rank,Company,Country,Industry,Revenue in USD
0,1,Walmart,United States,Retail,$500 billion
1,2,State Grid,China,Power,$349 billion
2,3,Sinopec Group,China,Petroleum,$327 billion
3,4,China National Petroleum,China,Petroleum,$326 billion
4,5,Royal Dutch Shell,Netherlands,Petroleum,$312 billion
5,6,Toyota Motor,Japan,Automobiles,$265 billion
6,7,Volkswagen,Germany,Automobiles,$260 billion
7,8,BP,United Kingdom,Petroleum,$245 billion
8,9,Exxon Mobil,United States,Petroleum,$244 billion
9,10,Berkshire Hathaway,United States,Products,$242 billion


In [65]:
# def convert_revenue(x):
#     return float(x.lstrip('$').rstrip('billion')) * 1e9

In [71]:
df['Revenue in USD'].apply(convert_revenue)

0    5.000000e+11
1    3.490000e+11
2    3.270000e+11
3    3.260000e+11
4    3.120000e+11
5    2.650000e+11
6    2.600000e+11
7    2.450000e+11
8    2.440000e+11
9    2.420000e+11
Name: Revenue in USD, dtype: float64

In [73]:
mapping

{'Apple': 'Apple Inc.',
 'BP': 'BP plc',
 'Berkshire Hathaway': 'Berkshire Hathaway Inc.',
 'China National Petroleum': 'China National Petroleum Corporation',
 'Exxon Mobil': 'Exxon Mobile Corporation',
 'Sinopec Group': 'China Petrochemical Corporation',
 'State Grid': 'State Grid Corporation of China',
 'Toyota Motor': 'Toyota Motor Corporation',
 'Volkswagen': 'Volkswagen AG'}

In [75]:
dict_head(dict_data, 10)

{'Walmart': {'name': 'Walmart', 'employees': 2300000, 'year founded': 1962},
 'State Grid Corporation of China': {'name': 'State Grid Corporation of China',
  'employees': 927839,
  'year founded': 2002},
 'China Petrochemical Corporation': {'name': 'China Petrochemical Corporation',
  'employees': 358571,
  'year founded': 1998},
 'China National Petroleum Corporation': {'name': 'China National Petroleum Corporation',
  'employees': 1636532,
  'year founded': 1988},
 'Toyota Motor Corporation': {'name': 'Toyota Motor Corporation',
  'employees': 364445,
  'year founded': 1937},
 'Volkswagen AG': {'name': 'Volkswagen AG',
  'employees': 642292,
  'year founded': 1937},
 'Royal Dutch Shell': {'name': 'Royal Dutch Shell',
  'employees': 92000,
  'year founded': 1907},
 'Berkshire Hathaway Inc.': {'name': 'Berkshire Hathaway Inc.',
  'employees': 377000,
  'year founded': 1839},
 'Apple Inc.': {'name': 'Apple Inc.',
  'employees': 123000,
  'year founded': 1976},
 'Exxon Mobile Corporatio

In [76]:
df['rev'] = df['Revenue in USD'].apply(convert_revenue)
df['employees'] = df['Company'].apply(lambda x : dict_data[mapping.get(x, x)].get('employees'))
df['rev_per_employee'] = df['rev'] / df['employees'].astype(float)
df.sort_values(by='rev_per_employee', ascending=False)

,Rank,Company,Country,Industry,Revenue in USD,rev,employees,rev_per_employee
8,9,Exxon Mobil,United States,Petroleum,$244 billion,2.440000e+11,69600,3.505747e+06
4,5,Royal Dutch Shell,Netherlands,Petroleum,$312 billion,3.120000e+11,92000,3.391304e+06
7,8,BP,United Kingdom,Petroleum,$245 billion,2.450000e+11,74000,3.310811e+06
2,3,Sinopec Group,China,Petroleum,$327 billion,3.270000e+11,358571,9.119533e+05
5,6,Toyota Motor,Japan,Automobiles,$265 billion,2.650000e+11,364445,7.271330e+05
9,10,Berkshire Hathaway,United States,Products,$242 billion,2.420000e+11,377000,6.419098e+05
6,7,Volkswagen,Germany,Automobiles,$260 billion,2.600000e+11,642292,4.048003e+05
1,2,State Grid,China,Power,$349 billion,3.490000e+11,927839,3.761428e+05
0,1,Walmart,United States,Retail,$500 billion,5.000000e+11,2300000,2.173913e+05
3,4,China National Petroleum,China,Petroleum,$326 billion,3.260000e+11,1636532,1.992017e+05


In [80]:
df_list = pd.read_html("https://en.wikipedia.org/w/index.php?title=Automotive_industry&oldid=875776152", header=0)
df_totals = df_list[0]
df_by_man = df_list[2]

In [83]:
df_totals

,Year,Production,Change,Source
0,1997,54434000,—,[17]
1,1998,52987000,2.7%,[17]
2,1999,56258892,6.2%,[18]
3,2000,58374162,3.8%,[19]
4,2001,56304925,3.5%,[20]
5,2002,58994318,4.8%,[21]
6,2003,60663225,2.8%,[22]
7,2004,64496220,6.3%,[23]
8,2005,66482439,3.1%,[24]
9,2006,69222975,4.1%,[25]


In [88]:
df_totals.sort_values(by='Year').iloc[-1]['Production']

97302534

In [90]:
total_prod = df_totals.sort_values(by='Year').iloc[-1]['Production']
total_prod

97302534

In [93]:
df_by_man

,Rank,Group,Country,Vehicles
0,1,Toyota,Japan,10213486
1,2,Volkswagen Group,Germany,10126281
2,3,Hyundai,South Korea,7889538
3,4,General Motors,United States,7793066
4,5,Ford,United States,6429485
5,6,Nissan,Japan,5556241
6,7,Honda,Japan,4999266
7,8,Fiat Chrysler Automobiles,Italy / United States,4681457
8,9,Renault,France,3373278
9,10,PSA,France,3152787


In [95]:
sum(df_by_man.Vehicles)

76329050

In [100]:
df_by_man['share'] = df_by_man['Vehicles'].astype(float) / total_prod
market_share = df_by_man.set_index('Group')['share'][['Toyota', 'Volkswagen Group']]
market_share

Group
Toyota              0.104966
Volkswagen Group    0.104070
Name: share, dtype: float64

In [124]:
rev = pd.read_html("https://en.wikipedia.org/w/index.php?title=List_of_largest_oil_and_gas_companies_by_revenue&oldid=871711850", header=1)[0]
rev = rev.iloc[:, 1:3]
rev.columns = ['Company', 'Revenue']
rev = rev[~(rev['Company'] == 'Company name')]

print(rev.shape)
rev
rev

(48, 2)


,Company,Revenue
0,Saudi Aramco,465.49
1,Sinopec Group,448.00
2,China National Petroleum Corporation,428.62
3,Exxon Mobil,268.9
4,Royal Dutch Shell,265
5,Kuwait Petroleum Corporation,251.94
6,BP,222.8
7,Total SA,212
8,Lukoil,144.17
9,Eni,131.82


In [125]:
rev['rev_clean'] = rev['Revenue'].apply(lambda x : float(x.rstrip('*')))
total = rev['rev_clean'].sum()
total


5482.09

In [126]:
rev['share'] = rev['rev_clean'] / total
rev

,Company,Revenue,rev_clean,share
0,Saudi Aramco,465.49,465.49,0.084911
1,Sinopec Group,448.00,448.00,0.081721
2,China National Petroleum Corporation,428.62,428.62,0.078186
3,Exxon Mobil,268.9,268.90,0.049051
4,Royal Dutch Shell,265,265.00,0.048339
5,Kuwait Petroleum Corporation,251.94,251.94,0.045957
6,BP,222.8,222.80,0.040641
7,Total SA,212,212.00,0.038671
8,Lukoil,144.17,144.17,0.026298
9,Eni,131.82,131.82,0.024046


In [128]:
mask = rev['Company'].isin(['Exxon Mobil', 'Sinopec Group', 'China National Petroleum Corporation', 'Royal Dutch Shell'])

mask

0     False
1      True
2      True
3      True
4      True
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
Name: Company, dtype: bool

In [129]:
rev[mask]

,Company,Revenue,rev_clean,share
1,Sinopec Group,448.00,448.00,0.081721
2,China National Petroleum Corporation,428.62,428.62,0.078186
3,Exxon Mobil,268.9,268.90,0.049051
4,Royal Dutch Shell,265,265.00,0.048339


In [130]:
rev = rev[mask].copy()
rev

,Company,Revenue,rev_clean,share
1,Sinopec Group,448.00,448.00,0.081721
2,China National Petroleum Corporation,428.62,428.62,0.078186
3,Exxon Mobil,268.9,268.90,0.049051
4,Royal Dutch Shell,265,265.00,0.048339


In [131]:
# do fuzzy search
split_names = {i: set(i.split(' ')) for i in df['Company']}

def fuzzy(word):
    return fuzzy_match(word, split_names)

rev['name'] = rev['Company'].apply(fuzzy)
rev

,Company,Revenue,rev_clean,share,name
1,Sinopec Group,448.00,448.00,0.081721,Sinopec Group
2,China National Petroleum Corporation,428.62,428.62,0.078186,China National Petroleum
3,Exxon Mobil,268.9,268.90,0.049051,Exxon Mobil
4,Royal Dutch Shell,265,265.00,0.048339,Royal Dutch Shell


In [132]:
ms2 = df.merge(rev[['share', 'name']], left_on='Company', right_on='name')
ms2

,Rank,Company,Country,Industry,Revenue in USD,rev,employees,rev_per_employee,share,name
0,3,Sinopec Group,China,Petroleum,$327 billion,3.270000e+11,358571,9.119533e+05,0.081721,Sinopec Group
1,4,China National Petroleum,China,Petroleum,$326 billion,3.260000e+11,1636532,1.992017e+05,0.078186,China National Petroleum
2,5,Royal Dutch Shell,Netherlands,Petroleum,$312 billion,3.120000e+11,92000,3.391304e+06,0.048339,Royal Dutch Shell
3,9,Exxon Mobil,United States,Petroleum,$244 billion,2.440000e+11,69600,3.505747e+06,0.049051,Exxon Mobil


Now we want to put these together and get only the company and the market share.

In [133]:
ms = market_share.reset_index()[['Group','share']]
ms.columns = ['Company', 'share']
pd.concat([ms, ms2[['Company', 'share']]])

,Company,share
0,Toyota,0.104966
1,Volkswagen Group,0.104070
0,Sinopec Group,0.081721
1,China National Petroleum,0.078186
2,Royal Dutch Shell,0.048339
3,Exxon Mobil,0.049051


*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*